#Question Answering with BERT
##AI VIETNAM
**Nguyễn Quốc Thái**
**thai.nq07@gmail.com**

In [ ]:
!nvidia-smi

Sun Mar  5 08:08:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    51W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install transformers==4.26.1 datasets==2.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.1 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/AI_VIETNAM/Course_2022/NLP/question_answering/code

/content/drive/MyDrive/AI_VIETNAM/Course_2022/NLP/question_answering/code


In [ ]:
import os
import urllib
import numpy as np
import pandas as pd
import torch
import datasets
from torch.utils.data import Dataset, DataLoader
from transformers import *

/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


#Using SQuAD dataset

##1.Load Data

In [ ]:
def load_data(data_url, num_sample=None):
    response = urllib.request.urlopen(data_url)
    df = pd.read_json(response)
    contexts = []
    questions = []
    answers = []
    ids = []
    for i in range(df['data'].shape[0]):
        paragraphs =df['data'].iloc[i]['paragraphs']
        for paragraph in paragraphs:
            context = paragraph['context']
            for q_a in paragraph['qas']:
                questions.append(q_a['question'])
                contexts.append(context)
                ids.append(q_a['id'])
                if q_a['is_impossible'] is False:
                    answers.append(q_a['answers'][0])
                else:
                    answer = {}
                    answer['answer_start'] = 0
                    answer['text'] = 'no-answer'
                    answers.append(answer)
    if num_sample is None:
        return contexts, questions, answers, ids
    else:
        return contexts[:500], questions[:500], answers[:500], ids[:500]


def add_end_idx(contexts, answers):
    for answer, context in zip(answers, contexts):
        if answer['text'] == 'no-answer':
            answer['answer_end'] = 0
        else:
            text = answer['text']
            start_idx = answer['answer_start']
            end_idx = start_idx + len(text)
            
            if context[start_idx:end_idx] == text:
                answer['answer_end'] = end_idx
            elif context[start_idx-1:end_idx-1] == text:
                answer['answer_start'] = start_idx - 1
                answer['answer_end'] = end_idx - 1
            elif context[start_idx-2: end_idx-2] == text:
                answer['answer_start'] = start_idx - 2
                answer['answer_end'] = end_idx - 2
    return contexts, answers

VD: tôi đi học start 10, 30 => 10, 31

def add_token_positions(encodings, answers, tokenizer):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        if answers[i]['text'] == 'no-answer':
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_positions.append(encodings.char_to_token(i,answers[i]['answer_start']))
            end_positions.append(encodings.char_to_token(i,answers[i]['answer_end']-1))
            #if none, the answer span has been truncated
            if start_positions[-1] is None:
                start_positions[-1] = tokenizer.model_max_length
            if end_positions[-1] is None:
                end_positions[-1] = tokenizer.model_max_length
    
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return encodings, start_positions, end_positions

tôi đi học => tôi => 3
t _ôi => 2

##2.Dataset

In [ ]:
class QADataset(Dataset):
    def __init__(self, cfg, data_url):
        self.cfg = cfg
        if cfg.is_debug:
            self.contexts, self.questions, self.answers, self.ids = load_data(data_url, num_sample=500)
        else:
            self.contexts, self.questions, self.answers, self.ids = load_data(data_url)

        self.contexts, self.answers = add_end_idx(self.contexts, self.answers)
        encodings = self.cfg.tokenizer(
            self.contexts,
            self.questions,  
            truncation = True,
            padding='max_length', 
            max_length=self.cfg.max_len,
            return_tensors='pt'
        )

        self.encodings, self.start_positions, self.end_positions = add_token_positions(
            encodings, 
            self.answers, 
            self.cfg.tokenizer)
    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings['input_ids'][idx],
            "attention_mask": self.encodings['attention_mask'][idx],
            "start_positions": self.start_positions[idx],
            "end_positions": self.end_positions[idx],
        }

    def __len__(self):
        return np.shape(self.encodings['input_ids'])[0]

##3.Metric

In [ ]:
metric = datasets.load_metric("squad_v2")

def compute_metrics(eval_pred):
    logits, labels = eval_pred      # predictions, label_ids (input_ids, start_positions, end_positions)
    start_logits, end_logits = logits
    input_ids = valid_dataset.encodings['input_ids']
    valid_answers = valid_dataset.answers

    predictions = []
    references = []
    for idx, (input_id, start_logit, end_logit, valid_answer) in enumerate(
        zip(input_ids, start_logits, end_logits, valid_answers)):

        input_id = np.delete(input_id, np.where(input_id == cfg.tokenizer.pad_token_id))
        
        start_prediction = np.argmax(start_logit)
        end_prediction = np.argmax(end_logit)
        span_prediction = input_id[start_prediction : end_prediction+1]

        answer_pred = cfg.tokenizer.decode(span_prediction, skip_special_tokens=True)

        # predictions.append(answer_pred)
        answer_ref = valid_answer['text']
        if answer_ref == 'no-answer':
            answer_ref = ''

        predictions.append({
            'prediction_text': answer_pred, 
            'id': str(idx), 
            'no_answer_probability': 0.
        })

        references.append({
            'answers': {
                'answer_start': [start_prediction], 
                'text': [answer_ref]
                }, 
            'id': str(idx)
        })

    score = metric.compute(predictions=predictions, references=references)
    return score

<ipython-input-10-981eb1b9fe0a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("squad_v2")


In [ ]:
metric = datasets.load_metric("squad_v2")

def predict_from_trainer(eval_pred):
    logits, labels = eval_pred      # predictions, label_ids (input_ids, start_positions, end_positions)
    start_logits, end_logits = logits
    input_ids = valid_dataset.encodings['input_ids']

    predictions = []
    for idx, (input_id, start_logit, end_logit) in enumerate(
        zip(input_ids, start_logits, end_logits)):

        input_id = np.delete(input_id, np.where(input_id == cfg.tokenizer.pad_token_id))
        
        start_prediction = np.argmax(start_logit)
        end_prediction = np.argmax(end_logit)
        span_prediction = input_id[start_prediction : end_prediction+1]

        answer_pred = cfg.tokenizer.decode(span_prediction, skip_special_tokens=True)

        predictions.append(answer_pred)
    return predictions

def load_valid_reference(cfg):
    response = urllib.request.urlopen(cfg.valid_url)
    data_json = json.loads(response.read())
    data = data_json['data']
    return data

In [ ]:
"""Official evaluation script for SQuAD version 2.0.

In addition to basic functionality, we also compute additional statistics and
plot precision-recall curves if an additional na_prob.json file is provided.
This file is expected to map question ID's to the model's predicted probability
that a question is unanswerable.
"""
import argparse
import collections
import json
import numpy as np
import os
import re
import string
import sys

def make_qid_to_has_ans(dataset):
    qid_to_has_ans = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid_to_has_ans[qa['id']] = bool(qa['answers'])
    return qid_to_has_ans

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s: return []
    return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def get_raw_scores(dataset, preds):
    exact_scores = {}
    f1_scores = {}
    for article in dataset:
        for p in article['paragraphs']:
            for qa in p['qas']:
                qid = qa['id']
                gold_answers = [a['text'] for a in qa['answers']
                                if normalize_answer(a['text'])]
                if not gold_answers:
                    # For unanswerable questions, only correct answer is empty string
                    gold_answers = ['']
                if qid not in preds:
                    print('Missing prediction for %s' % qid)
                    continue
                a_pred = preds[qid]
                # Take max over all gold answers
                exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
                f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
    return exact_scores, f1_scores

def apply_no_ans_threshold(scores, na_probs, qid_to_has_ans, na_prob_thresh):
    new_scores = {}
    for qid, s in scores.items():
        pred_na = na_probs[qid] > na_prob_thresh
        if pred_na:
            new_scores[qid] = float(not qid_to_has_ans[qid])
        else:
            new_scores[qid] = s
    return new_scores

def make_eval_dict(exact_scores, f1_scores, qid_list=None):
    if not qid_list:
        total = len(exact_scores)
        return collections.OrderedDict([
            ('exact', 100.0 * sum(exact_scores.values()) / total),
            ('f1', 100.0 * sum(f1_scores.values()) / total),
            ('total', total),
        ])
    else:
        total = len(qid_list)
        return collections.OrderedDict([
            ('exact', 100.0 * sum(exact_scores[k] for k in qid_list) / total),
            ('f1', 100.0 * sum(f1_scores[k] for k in qid_list) / total),
            ('total', total),
        ])

def merge_eval(main_eval, new_eval, prefix):
    for k in new_eval:
        main_eval['%s_%s' % (prefix, k)] = new_eval[k]

def compute_score(OPTS):
    OPTS.na_prob_file = None
    OPTS.na_prob_thresh = 1.0
    OPTS.out_file = None

    dataset = OPTS.reference
    preds = OPTS.preds
    if OPTS.na_prob_file:
        with open(OPTS.na_prob_file) as f:
            na_probs = json.load(f)
    else:
        na_probs = {k: 0.0 for k in preds}
    cfg.na_prob_file = None
    OPTS = cfg
    OPTS.na_prob_thresh = 1.0
    OPTS.out_file = None
    qid_to_has_ans = make_qid_to_has_ans(dataset)  # maps qid to True/False
    has_ans_qids = [k for k, v in qid_to_has_ans.items() if v]
    no_ans_qids = [k for k, v in qid_to_has_ans.items() if not v]
    exact_raw, f1_raw = get_raw_scores(dataset, preds)
    exact_thresh = apply_no_ans_threshold(exact_raw, na_probs, qid_to_has_ans,
                                            OPTS.na_prob_thresh)
    f1_thresh = apply_no_ans_threshold(f1_raw, na_probs, qid_to_has_ans,
                                        OPTS.na_prob_thresh)
    out_eval = make_eval_dict(exact_thresh, f1_thresh)
    if has_ans_qids:
        has_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=has_ans_qids)
        merge_eval(out_eval, has_ans_eval, 'HasAns')
    if no_ans_qids:
        no_ans_eval = make_eval_dict(exact_thresh, f1_thresh, qid_list=no_ans_qids)
        merge_eval(out_eval, no_ans_eval, 'NoAns')
    return out_eval

##4.Model "bert-base-uncased"

###4.1.Config

In [ ]:
class BaseConfig:
    """ base Encoder Decoder config """

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class NMTConfig(BaseConfig):
    # Data
    max_len = 512

    train_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
    valid_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json"

    # Model
    model_name = "bert-base-uncased"
    save_model = './save_model/' + model_name

    # Training
    is_debug = False

cfg = NMTConfig()
cfg.tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
cfg.model = AutoModelForQuestionAnswering.from_pretrained(cfg.model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_ac

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_model.bin
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenc

###4.2.**Load** Dataset

In [ ]:
train_dataset = QADataset(cfg, cfg.train_url)
valid_dataset = QADataset(cfg, cfg.valid_url)

###4.3.Training

In [ ]:
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    cfg.save_model,
    num_train_epochs=5,
    learning_rate=1e-4,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_strategy="steps",
    evaluation_strategy="steps",
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    eval_steps=2000,
    save_steps=2000
    )

trainer = Trainer(
    model=cfg.model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 130319
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 20365
  Number of trainable parameters = 108893186


Step,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
2000,1.349000,1.293254,51.958225,58.682124,11873,51.958225,58.682124,11873,51.958225,0.000000,58.682124,0.000000
4000,1.154600,1.247094,58.603554,64.344472,11873,58.603554,64.344472,11873,58.603554,0.000000,64.344472,0.000000
6000,0.883500,1.204285,58.342458,64.960430,11873,58.342458,64.960430,11873,58.342458,0.000000,64.960430,0.000000
8000,0.859300,1.220999,59.311042,66.571313,11873,59.311042,66.571313,11873,59.311042,0.000000,66.571313,0.000000


***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/bert-base-uncased/checkpoint-2000
Configuration saved in ./save_model/bert-base-uncased/checkpoint-2000/config.json
Model weights saved in ./save_model/bert-base-uncased/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/bert-base-uncased/checkpoint-4000
Configuration saved in ./save_model/bert-base-uncased/checkpoint-4000/config.json
Model weights saved in ./save_model/bert-base-uncased/checkpoint-4000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/bert-base-uncased/checkpoint-6000
Configuration saved in ./save_model/bert-base-uncased/checkpoint-6000/config.json
Model weights saved in ./save_model/bert-base-uncased/checkpoint-6000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1187

Step,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
2000,1.349000,1.293254,51.958225,58.682124,11873,51.958225,58.682124,11873,51.958225,0.000000,58.682124,0.000000
4000,1.154600,1.247094,58.603554,64.344472,11873,58.603554,64.344472,11873,58.603554,0.000000,64.344472,0.000000
6000,0.883500,1.204285,58.342458,64.960430,11873,58.342458,64.960430,11873,58.342458,0.000000,64.960430,0.000000
8000,0.859300,1.220999,59.311042,66.571313,11873,59.311042,66.571313,11873,59.311042,0.000000,66.571313,0.000000
10000,0.583400,1.331065,61.905163,68.639317,11873,61.905163,68.639317,11873,61.905163,0.000000,68.639317,0.000000
12000,0.589800,1.229538,62.351554,69.319380,11873,62.351554,69.319380,11873,62.351554,0.000000,69.319380,0.000000
14000,0.348300,1.731480,59.841658,67.359403,11873,59.841658,67.359403,11873,59.841658,0.000000,67.359403,0.000000
16000,0.338000,1.470304,62.183105,69.311534,11873,62.183105,69.311534,11873,62.183105,0.000000,69.311534,0.000000
18000,0.183900,1.990312,60.709172,68.049209,11873,60.709172,68.049209,11873,60.709172,0.000000,68.049209,0.000000


***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/bert-base-uncased/checkpoint-10000
Configuration saved in ./save_model/bert-base-uncased/checkpoint-10000/config.json
Model weights saved in ./save_model/bert-base-uncased/checkpoint-10000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/bert-base-uncased/checkpoint-12000
Configuration saved in ./save_model/bert-base-uncased/checkpoint-12000/config.json
Model weights saved in ./save_model/bert-base-uncased/checkpoint-12000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/bert-base-uncased/checkpoint-14000
Configuration saved in ./save_model/bert-base-uncased/checkpoint-14000/config.json
Model weights saved in ./save_model/bert-base-uncased/checkpoint-14000/pytorch_model.bin
***** Running Evaluation *****
  Num exampl

KeyboardInterrupt: ignored

###4.4.Evaluate

In [ ]:
cfg.model = AutoModelForQuestionAnswering.from_pretrained(cfg.save_model + "/checkpoint-12000")

loading configuration file ./save_model/bert-base-uncased/checkpoint-12000/config.json
Model config BertConfig {
  "_name_or_path": "./save_model/bert-base-uncased/checkpoint-12000",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./save_model/bert-base-uncased/checkpoint-12000/pytorch_model.bin
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weight

In [ ]:
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    cfg.save_model,
    num_train_epochs=5,
    learning_rate=1e-4,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_strategy="steps",
    evaluation_strategy="steps",
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    eval_steps=2000,
    save_steps=2000
    )

trainer = Trainer(
    model=cfg.model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
valid_predictions = trainer.predict(valid_dataset)
predictions = predict_from_trainer((valid_predictions.predictions, valid_predictions.label_ids))

***** Running Prediction *****
  Num examples = 11873
  Batch size = 32


In [ ]:
cfg.reference = load_valid_reference(cfg)

In [ ]:
preds = {key: val for key, val in zip(valid_dataset.ids, predictions)}
cfg.preds = preds
compute_score(cfg)

OrderedDict([('exact', 67.89353996462562),
             ('f1', 72.24771114094355),
             ('total', 11873),
             ('HasAns_exact', 58.85627530364373),
             ('HasAns_f1', 67.57710431451106),
             ('HasAns_total', 5928),
             ('NoAns_exact', 76.9049621530698),
             ('NoAns_f1', 76.9049621530698),
             ('NoAns_total', 5945)])

##5.Model "xlm-roberta-base"

###5.1.Config

In [ ]:
class BaseConfig:
    """ base Encoder Decoder config """

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class NMTConfig(BaseConfig):
    # Data
    max_len = 512

    train_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
    valid_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json"

    # Model
    model_name = "xlm-roberta-base"
    save_model = './save_model/' + model_name

    # Training
    is_debug = False

cfg = NMTConfig()
cfg.tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
cfg.model = AutoModelForQuestionAnswering.from_pretrained(cfg.model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/pytorch_model.bin
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initial

###5.2.Load Dataset

In [ ]:
train_dataset = QADataset(cfg, cfg.train_url)
valid_dataset = QADataset(cfg, cfg.valid_url)

###5.3.Training


In [ ]:
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    cfg.save_model,
    num_train_epochs=5,
    learning_rate=1e-4,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_strategy="steps",
    evaluation_strategy="steps",
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    eval_steps=2000,
    save_steps=2000
    )

trainer = Trainer(
    model=cfg.model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 130319
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 20365
  Number of trainable parameters = 277454594


Step,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
2000,1.483300,1.220722,57.592858,62.540309,11873,57.592858,62.540309,11873,57.592858,0.000000,62.540309,0.000000
4000,1.245700,1.107977,60.540723,66.167029,11873,60.540723,66.167029,11873,60.540723,0.000000,66.167029,0.000000
6000,1.064500,1.116590,62.191527,68.636402,11873,62.191527,68.636402,11873,62.191527,0.000000,68.636402,0.000000
8000,0.991100,1.034290,63.505432,69.708075,11873,63.505432,69.708075,11873,63.505432,0.000000,69.708075,0.000000
10000,0.798200,1.212638,64.718268,70.764339,11873,64.718268,70.764339,11873,64.718268,0.000000,70.764339,0.000000
12000,0.769900,1.064985,68.129369,73.650727,11873,68.129369,73.650727,11873,68.129369,0.000000,73.650727,0.000000
14000,0.555200,1.317513,64.339257,71.070996,11873,64.339257,71.070996,11873,64.339257,0.000000,71.070996,0.000000
16000,0.542100,1.151578,67.379769,73.574147,11873,67.379769,73.574147,11873,67.379769,0.000000,73.574147,0.000000
18000,0.370600,1.394119,67.160785,73.811002,11873,67.160785,73.811002,11873,67.160785,0.000000,73.811002,0.000000
20000,0.364000,1.402487,66.992336,73.604268,11873,66.992336,73.604268,11873,66.992336,0.000000,73.604268,0.000000


***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/xlm-roberta-base/checkpoint-2000
Configuration saved in ./save_model/xlm-roberta-base/checkpoint-2000/config.json
Model weights saved in ./save_model/xlm-roberta-base/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/xlm-roberta-base/checkpoint-4000
Configuration saved in ./save_model/xlm-roberta-base/checkpoint-4000/config.json
Model weights saved in ./save_model/xlm-roberta-base/checkpoint-4000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11873
  Batch size = 32
Saving model checkpoint to ./save_model/xlm-roberta-base/checkpoint-6000
Configuration saved in ./save_model/xlm-roberta-base/checkpoint-6000/config.json
Model weights saved in ./save_model/xlm-roberta-base/checkpoint-6000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 11873
  Batch

TrainOutput(global_step=20365, training_loss=0.876201853856331, metrics={'train_runtime': 15294.2441, 'train_samples_per_second': 42.604, 'train_steps_per_second': 1.332, 'total_flos': 1.7025966020539392e+17, 'train_loss': 0.876201853856331, 'epoch': 5.0})

###5.4.Evaluate

In [ ]:
cfg.model = AutoModelForQuestionAnswering.from_pretrained(cfg.save_model + "/checkpoint-10000")

loading configuration file ./save_model/xlm-roberta-base/checkpoint-10000/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./save_model/xlm-roberta-base/checkpoint-10000",
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file ./save_model/xlm-roberta-base/checkpoint-10000/pytorch_model.bin
All model checkpoint weights were used when initial

In [ ]:
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    cfg.save_model,
    num_train_epochs=5,
    learning_rate=1e-4,
    warmup_ratio=0.05,
    weight_decay=0.01,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    save_strategy="steps",
    evaluation_strategy="steps",
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    eval_steps=2000,
    save_steps=2000
    )

trainer = Trainer(
    model=cfg.model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
valid_predictions = trainer.predict(valid_dataset)
predictions = predict_from_trainer((valid_predictions.predictions, valid_predictions.label_ids))

***** Running Prediction *****
  Num examples = 11873
  Batch size = 32


In [ ]:
cfg.reference = load_valid_reference(cfg)

In [ ]:
preds = {key: val for key, val in zip(valid_dataset.ids, predictions)}
cfg.preds = preds
compute_score(cfg)

OrderedDict([('exact', 73.8313821275162),
             ('f1', 77.14957219421001),
             ('total', 11873),
             ('HasAns_exact', 70.91767881241566),
             ('HasAns_f1', 77.56357467305247),
             ('HasAns_total', 5928),
             ('NoAns_exact', 76.7367535744323),
             ('NoAns_f1', 76.7367535744323),
             ('NoAns_total', 5945)])

#Test

In [ ]:
model_name = "xlm-roberta-base"
model_path = "./save_model/xlm-roberta-base/checkpoint-10000"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

Could not locate the tokenizer configuration file, will try to use the model config instead.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/42f548f32366559214515ec137cdd16002968bf6/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

In [ ]:
def predict(question, context):
    input = tokenizer.encode_plus(context, question, return_tensors='pt')
    res = model(**input)
    start_position = torch.argmax(res.start_logits[0])
    end_position = torch.argmax(res.end_logits[0])
    answer = tokenizer.decode(input['input_ids'][0][start_position:end_position+1], skip_special_tokens=True)
    return answer

In [ ]:
question = valid_dataset.questions[0]
context = valid_dataset.contexts[0]
predict(question, context)

'France'

In [ ]:
question = "Mỗi ngày có bao nhiêu ca mắc COVID-19 mới?"
context = "Theo thống kê, tổng số ca mắc COVID-19 từ ngày 22/1- 28/1 là 77 ca mắc, trung bình mỗi ngày có 11 ca mắc mới. Cả nước đã tiêm trên 266 triệu liều vaccine COVID-19, nhưng hàng chục tỉnh, thành vẫn đang tiêm chậm, thấp; Nhóm cố vấn kỹ thuật về sự tiến hóa virus SARS-CoV-2 (TAG-VE) của WHO đã nâng cấp độ cảnh báo đối với XBB.1.5."
predict(question, context)

'11'

In [ ]:
question = "Thủ đô Việt Nam ở đâu?"
context = "Theo thống kê, tổng số ca mắc COVID-19 từ ngày 22/1- 28/1 là 77 ca mắc, trung bình mỗi ngày có 11 ca mắc mới. Cả nước đã tiêm trên 266 triệu liều vaccine COVID-19, nhưng hàng chục tỉnh, thành vẫn đang tiêm chậm, thấp; Nhóm cố vấn kỹ thuật về sự tiến hóa virus SARS-CoV-2 (TAG-VE) của WHO đã nâng cấp độ cảnh báo đối với XBB.1.5."
predict(question, context)

''

In [ ]:
question = "Ai là người giàu nhất Việt Nam?"
context = """"Trong đó, khối tài sản của ông Phạm Nhật Vượng, người giàu nhất Việt Nam, giảm khoảng 2,2 tỷ USD. 
Từ đầu năm đến nay, cổ phiếu VIC của Vingroup đã giảm gần một nửa, chốt phiên giao dịch 26/12 ở mức gần 53.000 đồng mỗi đơn vị. 
Hiện tại, ông Vượng chỉ sở hữu khối tài sản trị giá 4 tỷ USD – mức thấp nhất kể từ năm 2017.
"""
predict(question, context)

'Phạm Nhật Vượng'